# Create Your Own Agent
In this section we lay out a template that you can use to create your very own agent!

#### Ensure the latest version of boto3 is shown below

In [ ]:
!pip freeze | grep boto3

#### Retrieve imports environment variable and bring libraries into notebook

#### Load in environment variables to notebook

In [ ]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model
%store -r supervisor_agent_foundation_model

In [ ]:
%run $IMPORTS_PATH

# Prerequisites

This notebook assumes that you have deployed the CloudFormation stack located at https://github.com/aws-samples/amazon-bedrock-agents-cancer-biomarker-discovery to your AWS account in workshop mode.

# Bedrock Agent Creation Flow

1. Instantiate Bedrock Agent with key characteristics (name, description, instructions, etc.)
2. Extract relevant information about agent (ARN, ID, alias, etc.)
3. Associate Knowledge Base with agent
4. Create Lambda function
5. Define function/API schema for agent
6. Add needed ActionGroups with associated Lambda functions to agent
7. Give agent permission to use needed Lambda function
8. Test sample query by invoking agent directly
9. Create official alias version ready for use by a supervisor agent

# Agent Creation
In this section we create a sample sub-agent

#### Fill out your agent configuration below 

In [ ]:
agent_name = '[INSERT AGENT NAME]'
agent_description = '[INSERT AGENT DESCRIPTION]'
agent_instruction = """[INSERT AGENT INSTRUCTION]"""

#### Instantiate your agent with the desired configuration

In [ ]:
agents = AgentsForAmazonBedrock()

sample_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

sample_agent

#### Extract useful agent information

In [ ]:
sample_agent_id = sample_agent[0]
sample_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{sample_agent_id}"

sample_agent_id, sample_agent_arn

#### Associate Knowledge Base with agent if desired
##### Simply provide the necessary KB ID in the cell below or skip this section

In [ ]:
kb_id = "XXXXXXXX"

In [ ]:
agents.associate_kb_with_agent(
    sample_agent_id,
    "[INSERT KNOWLEDGE BASE DESCRIPTION]",
    kb_id
)

#### Define the API Schema needed for an ActionGroup
##### A simple sample schema is provided below

In [ ]:
api_schema_string = '''{
  [INSERT OPENAPI SCHEMA HERE]
}'''

api_schema = {"payload": api_schema_string}

#### Create Lambda function for use by agent ActionGroup

In [ ]:
%%writefile sample_lambda.py
import json

def lambda_handler(event, context):
    '''[INSERT LAMBDA FUNCTION CODE]'''

#### Attach Lambda function and create ActionGroup

In [ ]:
# Define Lambda func. details
sample_lambda_function_name = "[INSERT LAMBDA FUNCTION NAME]"
sample_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{sample_lambda_function_name}"

In [ ]:
agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=sample_lambda_function_name,
    source_code_file="sample_lambda.py",
    agent_action_group_name="sampleActionGroup",
    agent_action_group_description="[INSERT ACTION GROUP DESCRIPTION HERE]",
    api_schema=api_schema,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
lambda_client = boto3.client('lambda', region)

# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": sample_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": sample_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=sample_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=sample_lambda_function_arn,
    StatementId="AllowSampleAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=sample_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Invoke Sample Agent Test Alias to see that it answers question properly

In [ ]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id:str = str(uuid.uuid1())

test_query = "[INSERT TEST QUERY HERE]"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=sample_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Now that agent has been tested via direct invoke, prepare it by creating an alias

In [ ]:
sample_agent_alias_id, sample_agent_alias_arn = agents.create_agent_alias(
    sample_agent[0], 'v1'
)

sample_agent_alias_id, sample_agent_alias_arn